In [1]:
import math
import numpy as np
from BlackScholes import BlackScholes
from digital_quanto_pricer import *
from Simulator import Simulator

In [2]:
spot = 106.6
strike = 110.
tau = 0.3
q = 0.
rs = 0.04
rf = 0.05
sigma_s = 0.4
sigma_f = 0.08
rho = -0.1
quanto_factor = 4

In [3]:
time_steps = 1000
dt = tau / time_steps
num_paths = 100000

## Sanity Check using Pseudo Random Number
### vanilla option

In [4]:
%%time
# payoffs = spot * np.exp(np.cumsum((rf - q - 0.5 * sigma_s ** 2) * dt + sigma_s * math.sqrt(dt) * np.random.standard_normal((time_steps, num_paths)), axis=0))
payoffs = Simulator.simulate_pseudo(spot, rf, q, sigma_s, dt, num_paths, time_steps)

CPU times: user 20.7 s, sys: 26.2 s, total: 46.8 s
Wall time: 1min 16s


In [5]:
x = [payoff[-1] for payoff in payoffs]
x = [max(0, item - strike) for item in x]
np.mean(x) * np.exp(-rf * tau)

8.546673276342952

In [6]:
opt = BlackScholes(spot, strike, rf, q, sigma_s, tau, BlackScholes.TAG_CALL)

In [7]:
opt.price()

8.520497276304305

In [8]:
opt.n_d2 * math.exp(-rf * tau)

0.42050044479801657

### Floating rate quanto for call option:

In [10]:
FloatingFXDigitalQuantoCall(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.42050044479801657

In [11]:
x = [1 if item > 0 else 0 for item in x]
np.mean(x)

0.427545

## Sanity Check using Sobol Random Number
### vanilla option

In [4]:
%%time
payoffs = Simulator.simulate_sobol(spot, rf, q, sigma_s, dt, num_paths, time_steps)

CPU times: user 6.86 s, sys: 3.03 s, total: 9.89 s
Wall time: 8.68 s


In [5]:
x = [payoff[-1] for payoff in payoffs]
x = [max(0, item - strike) for item in x]
np.mean(x) * np.exp(-rf * tau)

8.52001030769462

In [6]:
opt = BlackScholes(spot, strike, rf, q, sigma_s, tau, BlackScholes.TAG_CALL)

In [7]:
opt.price()

8.520497276304305

In [8]:
opt.n_d2 * math.exp(-rf * tau)

0.42050044479801657

### Floating Rate Digital Quanto Call:
#### Denominated in fx price (vanilla digital)

In [9]:
FloatingFXDigitalQuantoCall(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.42050044479801657

In [10]:
x = [1 if item > 0 else 0 for item in x]
np.mean(x) * math.exp(-rf * tau)

0.42103684298634897

### Floating Rate Digital Quanto Put:
#### Denominated in fx price (vanilla digital):

In [11]:
FloatingFXDigitalQuantoPut(rs, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, 4).fx_price()

0.564611494805046

In [12]:
math.exp(-rf * tau) * (1 - np.mean(x))

0.5640750966167136